In [20]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import requests
import gmaps
import config

In [25]:
from config import g_key

ImportError: cannot import name 'g_key' from 'config' (/Users/lorenzocastillo/Desktop/World_Weather_Analysis/config.py)